# dev branch test


In [1]:
using BenchmarkTools


In [2]:
#include("fileio.jl")
#include("kabsch_umeyama.jl")
include("foldexity.jl")


In [3]:
pdb = readpdb("testpdb/small.pdb")
writepdb(pdb, "testpdb/small_backbone.pdb")

xyzcoords = pdb2matrix(pdb)
coordmatrix = matrix2fragments(xyzcoords)

pdbmatrix = pdb2pdbmatrix(pdb)
pdbfragments = matrix2fragments(pdbmatrix)

println()


In [4]:
pfrag1 = coordmatrix[10]
pfrag2 = coordmatrix[31]

align_rmsd(pfrag1, pfrag2)

0.39434291345455486

In [5]:
function kabsh_matrix(megax)    
    try        
        n = size(megax)[1]  # Change this to the desired size
        matrix = zeros(Float64, n, n)

        for i = 1:n # Fill the upper triangle
            for j = i+1:n  # Ensure j >= i for the upper triangle
                matrix[i, j] = align_rmsd(megax[i], megax[j])
            end
        end
        return sum(matrix) * 2 / n , matrix
        
    catch 
        return 0
    end
end

kabsh_matrix (generic function with 1 method)

In [6]:
fx, m = kabsh_matrix(coordmatrix)

(103.81768186522714, [0.0 1.0179540814779788 … 6.218812751325102 6.267977511576181; 0.0 0.0 … 6.319147071237217 6.365847933859438; … ; 0.0 0.0 … 0.0 0.23754784309650565; 0.0 0.0 … 0.0 0.0])

In [7]:
fxity, nfrags = fxpdb("testpdb/small.pdb")  

((103.81768186522714, [0.0 1.0179540814779788 … 6.218812751325102 6.267977511576181; 0.0 0.0 … 6.319147071237217 6.365847933859438; … ; 0.0 0.0 … 0.0 0.23754784309650565; 0.0 0.0 … 0.0 0.0]), 47)

In [8]:
data = fxdir("testpdb/", true)  

1     testpdb/3EFM.pdb        (2544.1938044136273, [0.0 3.351162755475926 2.814786266319581 4.185344290200501 3.3991093034038546 4.583456033739365 3.395159532837405 3.595265511133775 3.1801188060400256 3.580596853752544 3.010316886782014 3.632611893638659 4.149918311233257 2.9998465020990186 4.5250272682774435 3.9183203389654877 3.4863723059792395 5.017983045539923 5.092331431134598 3.893071182556955 3.101693612929217 2.4118171168529097 2.841058112984698 3.621455339295662 3.6452481950373907 5.877930008272148 5.563986046905931 2.876622655843081 3.491617026242138 2.9900925590332212 2.37604590230033 2.988645453905562 4.737804639781632 4.467408075781381 4.203084459887646 3.9482277021315366 3.9734126783248236 5.904153713895756 5.712650810183626 5.8465903922417874 5.735811699145061 5.712214552843906 5.2241046272644205 3.114218059997675 3.9519572600804436 4.707013148036291 3.0461991494639675 3.195741087170949 5.69966752411093 5.8324015744341535 5.839151501990887 5.70762206476288 5.37989742248

Excessive output truncated after 524288 bytes.

5-element Vector{Any}:
 Any[1, "testpdb/3EFM.pdb", (2544.1938044136273, [0.0 3.351162755475926 … 4.431764308692129 4.420757261794529; 0.0 0.0 … 4.1664591887554305 4.177621558302379; … ; 0.0 0.0 … 0.0 1.438477404471372; 0.0 0.0 … 0.0 0.0]), 569]
 Any[2, "testpdb/AF-Q8VCK6-F1-model_v4.pdb", (870.1390365526634, [0.0 1.0179540814779788 … 3.1094262133232973 3.394179650219974; 0.0 0.0 … 3.308608661988175 3.5397114307876443; … ; 0.0 0.0 … 0.0 1.4251403263875766; 0.0 0.0 … 0.0 0.0]), 327]
 Any[3, "testpdb/polya.pdb", (20.089421178909642, [0.0 0.030534984248102026 … 0.5576647072303506 0.371213177486735; 0.0 0.0 … 0.025536698528295085 0.01989440992795823; … ; 0.0 0.0 … 0.0 0.035365932205024216; 0.0 0.0 … 0.0 0.0]), 58]
 Any[4, "testpdb/small.pdb", (103.81768186522714, [0.0 1.0179540814779788 … 6.218812751325102 6.267977511576181; 0.0 0.0 … 6.319147071237217 6.365847933859438; … ; 0.0 0.0 … 0.0 0.23754784309650565; 0.0 0.0 … 0.0 0.0]), 47]
 Any[5, "testpdb/small_backbone.pdb", (103.81768186522714